In [16]:
r = 16
lora_alpha = 16
random_state = 3407

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/DeepSeek-R1-Distill-Qwen-7B-unsloth-bnb-4bit",
    )

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = r,
    lora_alpha = lora_alpha,
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = random_state,
)

In [ ]:
from datasets import load_dataset, Dataset
dataset:  Dataset  = load_dataset("beautyyuyanli/deep-sink-data", split = "train") # type: ignore

In [ ]:
from rich import print
print(dataset)

In [ ]:
from jinja2 import Template
template =Template("""
<｜User｜>写一篇爆仓文学，主题：{{topic}}
<｜Assistant｜><think>\n{{think}}\n</think>\n\n{{content}}\n<｜end▁of▁sentence｜>
""".strip())
def map_fn(e):
    return {"text": template.render(topic=e["topic"], think=e["think"], content=e["content"])}
    
dt_text = dataset.map(map_fn, remove_columns=dataset.column_names)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dt_text,
    dataset_text_field = "text",
    # max_seq_length = max_seq_length,
    # dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 60,
        num_train_epochs = 10, # For longer training runs!
        save_strategy="epoch",
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = random_state,
        output_dir = "../../../output_model",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
from unsloth import unsloth_train
unsloth_train(trainer=trainer)

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

In [1]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="../../../output_model/checkpoint-140",
    )


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/yanli.yu/DeepSink/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.684 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.21it/s]
Unsloth 2025.1.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

test = """
<｜User｜>写一篇爆仓文学。不得夹杂英文。
<｜Assistant｜><think>
""".strip()
input_ids = tokenizer(
    test,
    return_tensors = "pt",
).to("cuda")
input_ids = input_ids["input_ids"]

In [7]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 4096, pad_token_id = tokenizer.eos_token_id)

我应该这样设计写作思路:首先,我选择一个在金融市场爆仓的事件作为故事的核心冲突,这能迅速抓住读者的注意力,并营造紧张的氛围。为了增强故事的真实感和代入感,我使用了精确的时间点(凌晨3:17)和具体的数字(200万保证金、杠杆倍数、账户余额)。故事中嵌入了金融行业的专业术语(CTO、杠杆、保证金、强平、流动性、合约、杠杆体系),这不仅提升了故事的专业性,也让主人公成为了故事的“试验品”,让读者更容易产生共鸣。

为了更有效地表达主人公的内心世界,我使用了第一人称叙述,并细腻地描写了他的身体反应(喉咙发紧、手抖、汗毛竖起)以及他的外部环境(凌晨三点、电梯里、咖啡店、手机没电),以此来增强故事的感染力。我设计了多个原因导致爆仓(市场崩盘、公司重组、杠杆体系错误),这反映了金融市场的复杂性和不可预测性,也突出了主人公的盲目自信和错误判断。结尾处,主人公还在咖啡店等待dehydration报警,这暗示了他可能面临更严重的后果,也留下了余韵,让读者回味思考。

为了提升故事的层次感,我穿插了主人公的回忆(三年前在杭州打工、买股票、> 开始积累经验和心理准备),并与外部事件(金融公司重组、政策变化)形成了呼应。这种前后对比,不仅强化了主题,也展现了主人公的悲剧性命运。我使用了对比手法,将主人公之前的贪婪与后来的绝望形成对比,将他与之前在杭州打工的朋友形成对比,将他与那些了解期权知识的“专家”形成对比,以此来突出个人在群体面前的渺小和最终的失败。

在情感表达上,我力求细致而真实。主人公从希望到焦虑再到绝望,我细致地描绘了他的身体反应和心理活动,例如,喉咙发紧、手抖、汗毛竖起等细节,力求让读者感受到他的痛苦和无奈。我设置了一些伏笔(朋友的谎言、市场崩盘、政策变化),来进一步加剧故事的悲剧色彩。结尾处,主人公等待dehydration报警,这不仅是一个生理层面的描绘,更是一种对金融风险不可抗拒性的隐喻。

我使用了象征手法,“dehydration报警”可以解读为一种无法摆脱的困境,“强平提示”则象征着金融体系对个体的控制。我运用了时间(凌晨三点、> )、具体的数字(200万、500元余额)以及专业的金融术语(期权、> 、杠杆体系错误),这些元素共同构建了故事的紧张和专业性。

总而言之,我力图通过紧凑的情节、细致的描写和多样的写作手法,刻画一个在金融市场中孤独、绝望和自我毁灭的故事